# 作成した教材をGitHubへバックアップします
---

ホームディレクトリで作成した教材をGitHubリポジトリへバックアップします。

# バックアップ用GitHubリポジトリの準備

* 既にバックアップ用GitHubリポジトリが作成済みの場合は、この項を読み飛ばして結構です。
* 初めて教材をGitHubリポジトリへバックアップする場合は、[GitHubリポジトリの準備/1. GitHubで新しいリポジトリを作成](03_PrepareGitHub.ipynb#create_github_repo) にしたがって、バックアップ用のGitHubリポジトリを作成して下さい。

# GitHubリポジトリへの接続情報を設定します

バックアップ先のGitHubリポジトリへの接続情報を設定します。
1. GitHubに接続するSSHキーを新たに登録する場合は、 [GitHubリポジトリの準備/2. SSHキーの登録](03_PrepareGitHub.ipynb#create_ssh_key)を使って作成して下さい。
2. [GitHubリポジトリの準備/3. GitHubリポジトリ接続情報の設定](03_PrepareGitHub.ipynb#connect_github_repo)を使って、バックアップ用のGitHubリポジトリの接続情報を設定して下さい。

# バックアップの準備

このNotebookを実行すると、ホームディレクトリ直下の下記のフォルダをGitHubリポジトリに保存します。<br/>
**バックアップしたいファイル/フォルダは、下記のフォルダに移動してください。**
* `textbook`
  + 教材Notebookおよび教材Notebookから呼び出すスクリプトなど
* `info`
  + 受講者が参照する参考資料など
* `draft`
  + 教材以外で保存して置きたいNotebook、その他ファイル

注） GitHubには `test` フォルダも保存されます。
* `test` フォルダは、この教材の自動テストに必要なファイルを保存するフォルダです。
* **`test` フォルダにはその他のファイルを格納しないで下さい。**

# その他にバックアップしたいファイル/フォルダがある場合

もし、`textbook`、`info`、`draft` 以外に保存したいファイルまたはフォルダがある場合は、`INCLUDES` にスペース区切りで列挙して下さい。
* `textbook`、`info`、`draft`、`test` フォルダは必ずバックアップに保存されます。
* 管理用Notebookフォルダ `tools` はバックアップに保存されません
```
%env INCLUDES=ext-textbook ext-data Sample.ipynb
```

**次のセルでバックアップに含めたいファイル/フォルダを指定して、セルを実行して下さい。**

In [ ]:
%env INCLUDES=

# GitHubにpushします

指定したGitHub repositoryに保存します。

```
%env COMMIT_MESSAGE=Add special notebook.
```

**次のセルで、commitメッセージを設定して下さい。**

In [ ]:
%env COMMIT_MESSAGE=

**次のセルを実行して、GitHubに保存して下さい。**

In [ ]:
%%bash

cd
comm -13 <(sort ~/.ssh/known_hosts) <(ssh-keyscan github.com 2>/dev/null | sort) >>~/.ssh/known_hosts 

IS_INSIDE_REPO=0
S="$(git rev-parse --is-inside-work-tree 2>/dev/null)"
[ $? -eq 0 -a "${S}" = "true" ] && IS_INSIDE_REPO=1

if [ ${IS_INSIDE_REPO} -ne 1 ]; then
  echo "ERROR: git repository not initialized." 1>&2
  exit 1
fi

if [ ! -z "${INCLUDES}" ]; then
  for item in ${INCLUDES}; do
    git add -A "${item}"
  done
fi

if [ -z "${COMMIT_MESSAGE}" ]; then
  COMMIT_MESSAGE="Backup of $(< /etc/courseware.configmap/fqdn)."
fi

git add -A ./textbook/ ./info/ ./draft/
git fetch --force origin
git merge -s recursive -X ours origin/master
git commit -m "${COMMIT_MESSAGE}"
git push --set-upstream origin master